In [1]:
#Importing dependencies
import numpy as np
import pandas as pd
from sqlalchemy import create_engine
from datetime import datetime

# Importing Data From SQLite Database

We needed to retrieve the clean data from the SQLite database ('fraud_detection.db') and load it into a Pandas DataFrame to prepare it for analysis in the neural network model.

In [62]:
#Creating SQLAlchemy engine and connecting to the SQLite database
engine = create_engine('sqlite:///database/fraud_detection.db')
connection = engine.connect()

#Creating the pandas dataframe by reading the SQL table
model_df = pd.read_sql_table('CleanFraudDetection', con=connection)

model_df

transaction_id      time       category  amount   age gender state  \
0               119106  07:55:28   shopping_pos    1.07  59.0      F    CT   
1               179292  14:05:16      kids_pets   94.99  41.0      M    OK   
2               540729  16:22:29      kids_pets   31.28  69.0      F    MO   
3               374360  10:44:50  gas_transport   73.06  41.0      M    OK   
4               314574  01:50:09       misc_net    9.99  52.0      F    IA   
...                ...       ...            ...     ...   ...    ...   ...   
111139          444284  12:42:49    food_dining    8.22  24.0      F    IA   
111140           89444  21:29:49  personal_care  205.69  35.0      F    OH   
111141          298536  01:05:31       misc_net  277.63  43.0      M    MA   
111142          301993  04:16:28   shopping_net   13.29  92.0      F    KY   
111143          354010  23:38:46    food_dining  100.46  39.0      M    WI   

        city_pop  is_fraud  
0           5438         0  
1           7163         0  
2            602         0  
3           7163         0  
4           2036         0  
...          ...       ...  
111139      1583         0  
111140      7646         0  
111141     47249         0  
111142       571         0  
111143      1360         1  

[111144 rows x 9 columns]

# Dropping and Transforming Columns

Once the dataset was loaded into a pandas dataframe, certain columns needed to be dropped and transformed for the model to be created. In this case, it was necessary that the column 'time' was transformed into a datetime object and binned so that the model could interpret it. In addition, the transaction id needed to be deleted since it was not a feature or a target. Lastly, we used the pd.get_dummies function to convert the categorical data into numeric.

In [63]:
#Checking the data types
model_df.dtypes

transaction_id      int64
time               object
category           object
amount            float64
age               float64
gender             object
state              object
city_pop            int64
is_fraud            int64
dtype: object

In [64]:
#Converting 'time' column to datetime format
#We have to convert the 'time' column to a string first before converting it to a datetime object
model_df['time'] = model_df['time'].astype(str)
model_df['time'] = pd.to_datetime(model_df['time'])

#Defining custom time bins based on hour of the day
#Bins are as follows: early morning (12am-6am), late morning (6am-12pm).\
#afternoon (12pm-4pm), evening (4pm-8pm), night (8pm-12am)
bins = [0, 6, 12, 16, 20, 24]
labels = ['early morning', 'late morning', 'afternoon', 'evening', 'night']

#Extracting hour of the day and categorize into bins
model_df['hour'] = model_df['time'].dt.hour
model_df['time_bins'] = pd.cut(model_df['hour'], bins=bins, labels=labels, right=False)

#Dropping the 'hour' column if not needed
model_df = model_df.drop(columns=['hour', 'time'])

model_df

transaction_id       category  amount   age gender state  city_pop  \
0               119106   shopping_pos    1.07  59.0      F    CT      5438   
1               179292      kids_pets   94.99  41.0      M    OK      7163   
2               540729      kids_pets   31.28  69.0      F    MO       602   
3               374360  gas_transport   73.06  41.0      M    OK      7163   
4               314574       misc_net    9.99  52.0      F    IA      2036   
...                ...            ...     ...   ...    ...   ...       ...   
111139          444284    food_dining    8.22  24.0      F    IA      1583   
111140           89444  personal_care  205.69  35.0      F    OH      7646   
111141          298536       misc_net  277.63  43.0      M    MA     47249   
111142          301993   shopping_net   13.29  92.0      F    KY       571   
111143          354010    food_dining  100.46  39.0      M    WI      1360   

        is_fraud      time_bins  
0              0   late morning  
1              0      afternoon  
2              0        evening  
3              0   late morning  
4              0  early morning  
...          ...            ...  
111139         0      afternoon  
111140         0          night  
111141         0  early morning  
111142         0  early morning  
111143         1          night  

[111144 rows x 9 columns]

In [65]:
#Dropping the transaction_id since it's not a feature for the model
model_df.drop(columns='transaction_id', inplace=True)
model_df

category  amount   age gender state  city_pop  is_fraud  \
0        shopping_pos    1.07  59.0      F    CT      5438         0   
1           kids_pets   94.99  41.0      M    OK      7163         0   
2           kids_pets   31.28  69.0      F    MO       602         0   
3       gas_transport   73.06  41.0      M    OK      7163         0   
4            misc_net    9.99  52.0      F    IA      2036         0   
...               ...     ...   ...    ...   ...       ...       ...   
111139    food_dining    8.22  24.0      F    IA      1583         0   
111140  personal_care  205.69  35.0      F    OH      7646         0   
111141       misc_net  277.63  43.0      M    MA     47249         0   
111142   shopping_net   13.29  92.0      F    KY       571         0   
111143    food_dining  100.46  39.0      M    WI      1360         1   

            time_bins  
0        late morning  
1           afternoon  
2             evening  
3        late morning  
4       early morning  
...               ...  
111139      afternoon  
111140          night  
111141  early morning  
111142  early morning  
111143          night  

[111144 rows x 8 columns]

In [66]:
#Transforming the categorical data into numeric
categorical_df = pd.get_dummies(model_df)
categorical_df.shape

(111144, 75)

# Separating the Lables and the Features

In [67]:
#Separating the labels and the features 
X = categorical_df.drop(["is_fraud"], axis='columns').values
y = categorical_df["is_fraud"].values

In [68]:
#Reviewing the series of lables
y

array([0, 0, 0, ..., 0, 0, 1], dtype=int64)

In [69]:
#Reviewing the features
X

array([[1.0700e+00, 5.9000e+01, 5.4380e+03, ..., 0.0000e+00, 0.0000e+00,
        0.0000e+00],
       [9.4990e+01, 4.1000e+01, 7.1630e+03, ..., 1.0000e+00, 0.0000e+00,
        0.0000e+00],
       [3.1280e+01, 6.9000e+01, 6.0200e+02, ..., 0.0000e+00, 1.0000e+00,
        0.0000e+00],
       ...,
       [2.7763e+02, 4.3000e+01, 4.7249e+04, ..., 0.0000e+00, 0.0000e+00,
        0.0000e+00],
       [1.3290e+01, 9.2000e+01, 5.7100e+02, ..., 0.0000e+00, 0.0000e+00,
        0.0000e+00],
       [1.0046e+02, 3.9000e+01, 1.3600e+03, ..., 0.0000e+00, 0.0000e+00,
        1.0000e+00]])

# Split the Data

In [70]:
#Importing dependencies 
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [71]:
#Splitting the data into training and testing sets. Random state of 1 will be assigned.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

In [72]:
#Creating a StandardScaler instance
scaler = StandardScaler()

#Fitting the StandardScaler
X_scaler = scaler.fit(X_train)

#Scaling the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# Compile and Train the Model

Using the training and testing data, we created a neural network model with three layers. We created our model using 80 nodes in the first layer and 30 nodes in the second. We used 100 epochs for optimal model performance.

In [73]:
#Importing dependencies 
import tensorflow as tf
from sklearn.metrics import confusion_matrix, classification_report, balanced_accuracy_score

In [74]:
#Defining the model - deep neural net
number_input_features = len(X_train[0])
hidden_nodes_layer1 =  80
hidden_nodes_layer2 = 30
nn = tf.keras.models.Sequential()

#First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

#Second hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer2, input_dim=number_input_features, activation="relu")
)

#Output layer
nn.add(
    tf.keras.layers.Dense(units=1, activation="sigmoid")
)

#Checking the structure of the model
nn.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 80)                6000      
                                                                 
 dense_7 (Dense)             (None, 30)                2430      
                                                                 
 dense_8 (Dense)             (None, 1)                 31        
                                                                 
Total params: 8461 (33.05 KB)
Trainable params: 8461 (33.05 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [75]:
#Compiling the model
nn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [76]:
#Fitting the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
2779/2779 [==============================] - 7s 2ms/step - loss: 0.0194 - accuracy: 0.9957
Epoch 2/100
2779/2779 [==============================] - 5s 2ms/step - loss: 0.0124 - accuracy: 0.9972
Epoch 3/100
2779/2779 [==============================] - 5s 2ms/step - loss: 0.0106 - accuracy: 0.9975
Epoch 4/100
2779/2779 [==============================] - 5s 2ms/step - loss: 0.0093 - accuracy: 0.9978
Epoch 5/100
2779/2779 [==============================] - 5s 2ms/step - loss: 0.0082 - accuracy: 0.9981
Epoch 6/100
2779/2779 [==============================] - 5s 2ms/step - loss: 0.0074 - accuracy: 0.9983
Epoch 7/100
2779/2779 [==============================] - 5s 2ms/step - loss: 0.0065 - accuracy: 0.9985
Epoch 8/100
2779/2779 [==============================] - 5s 2ms/step - loss: 0.0061 - accuracy: 0.9986
Epoch 9/100
2779/2779 [==============================] - 5s 2ms/step - loss: 0.0058 - accuracy: 0.9985
Epoch 10/100
2779/2779 [==============================] - 5s 2ms/step - l

In [82]:
#Making a prediction with the model
prediction = nn.predict(X_test)
prediction

695/695 [==============================] - 1s 1ms/step


array([[0.],
       [0.],
       [0.],
       ...,
       [0.],
       [0.],
       [0.]], dtype=float32)

# Model Evaluation

The model achieved 99.8% accuracy, signifying that it was able to correctly identify instances of fraud 99.8% of the time.

In [83]:
#Evaluating the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled, y_test, verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

695/695 - 1s - loss: 0.0212 - accuracy: 0.9983 - 751ms/epoch - 1ms/step
Loss: 0.021244585514068604, Accuracy: 0.9983354806900024


In [84]:
#Calculating other metrics of model performance

#This converts the continuous outcomes to binary (anything below a 0.5 is not fraud, and anything above a 0.5 is fraud)
threshold = 0.5
predicted_labels = (prediction > threshold).astype(int)

#Calculating Balanced Accuracy Score
#The Balanced accuracy is low since there are substantially more 0s than 1s 
balanced_acc = balanced_accuracy_score(y_test, predicted_labels)
print("Balanced Accuracy Score:", balanced_acc)

Balanced Accuracy Score: 0.5


In [ ]:
predicted_labels

In [85]:
#Calculating confusion matrix
conf_matrix = confusion_matrix(y_test, predicted_labels)
print("Confusion Matrix:")
print(conf_matrix)


Confusion Matrix:
[[22166     0]
 [   63     0]]


In [86]:
#Generating classification report
class_report = classification_report(y_test, predicted_labels)
print("Classification Report:")
print(class_report)

Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     22166
           1       0.00      0.00      0.00        63

    accuracy                           1.00     22229
   macro avg       0.50      0.50      0.50     22229
weighted avg       0.99      1.00      1.00     22229



C:\Users\Asher\anaconda3\envs\dev\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Asher\anaconda3\envs\dev\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Asher\anaconda3\envs\dev\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
